---

# **<h1 align ="left"><b>Find similar clothing items</b></h1>**
### **<h1 align ="left"><b>Generate Embeddings& Qdrant Vector Database</b></h1>**

---

In [2]:
#tensorflow conda
import ocifs
import numpy as np
import torch
import pandas as pd
import whisper
import json

from tqdm.notebook import tqdm
#from sentence_transformers import SentenceTransformer
import pickle
import ffmpeg
import ocifs
from json import loads, dumps
#from qdrant_client import models, QdrantClient
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

import glob
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
import gradio as gr

## **Start Ilias**

## **1 Create embeddings and push results to Vector Database**

In [3]:
#load data
dfx = pd.read_excel("./Q&A.xlsx")
dfx.head()

,MODULE,SCREEN,QUESTION,ANSWER
0,Configuration,AFUNC,How can I define the attributes or specificati...,The attributes that define the functionality o...
1,Configuration,AFUNC,How can I define an alternative function?,You can define the use of a function for more ...
2,Configuration,AFUNC,How can I assign a function to a non-serialize...,The Function-NSN relationship is configured on...
3,Configuration,AFUNC,Where can I define the different operational s...,The different operational statuses applicable ...
4,Configuration,AFUNC,How can I determine the different abilities ap...,The different abilities applicable to the asse...


In [4]:
#format to what Qdrant expects
texts = dfx.to_json(orient='records')
documents = loads(texts)

In [5]:
#show some recorrdcs
documents[0:3]

[{'MODULE': 'Configuration',
  'SCREEN': 'AFUNC',
  'QUESTION ': 'How can I define the attributes or specifications of a function?',
  'ANSWER': 'The attributes that define the functionality of a material are defined in the AFUNC screen, Tabpage Function, attributes frame.'},
 {'MODULE': 'Configuration',
  'SCREEN': 'AFUNC',
  'QUESTION ': 'How can I define an alternative function?',
  'ANSWER': 'You can define the use of a function for more than one configuration tree on the AFUNC screen, Alternates tabpage.'},
 {'MODULE': 'Configuration',
  'SCREEN': 'AFUNC',
  'QUESTION ': 'How can I assign a function to a non-serialized NSN?',
  'ANSWER': 'The Function-NSN relationship is configured on the AFUNC screen, Tabpage related part.'}]

In [6]:
#embedding generator
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('multi-qa-distilbert-cos-v1') # Model to create embeddings

In [7]:
#example of the format it should have

# documents = [
#   { "name": "The Time Machine", "description": "A man travels through time and witnesses the evolution of humanity.", "author": "H.G. Wells", "year": 1895 },
#   { "name": "Ender's Game", "description": "A young boy is trained to become a military leader in a war against an alien race.", "author": "Orson Scott Card", "year": 1985 },
#   { "name": "Brave New World", "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.", "author": "Aldous Huxley", "year": 1932 }
# ]

In [8]:
#establish connection to Qdrant vector database
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client import AsyncQdrantClient, models

qdrant = QdrantClient("207.211.188.211", port=6333) # 

# Create collection to store
qdrant.recreate_collection(
    collection_name="collection_v3",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)


# vectorize descriptions and upload to qdrant
qdrant.upload_records(
    collection_name="collection_v3",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["QUESTION "]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(documents)
    ]
)

## **2 Query the Vector Database**

In [9]:
## example Search on a sentence
hits = qdrant.search(
    collection_name="collection_v3",
    query_vector=encoder.encode("I want to change the configuration tree").tolist(),
    limit=10
)

list_of_answers = []

x = 1
for hit in hits:
    #print(hit.payload, "score:", hit.score)
    print("---- Result ---- " + str(x))
    output = hit.payload
    module = output['MODULE']
    screen = output['SCREEN']
    answer = output['ANSWER']
    question = output['QUESTION ']
    
    print("Module = " + module)
    print("Screen = " + screen)
    print("Answer = " + answer)
    print("Question = " + question)
    print("----------------")
    print()
    
    list_of_answers.append(answer)
    
    x += 1
    

---- Result ---- 1
Module = Operations
Screen = AMISFU
Answer = In the screen AINSTR you can installe and remove assets from a configuration tree.
Question = How can I remove assets from a configuration tree?
----------------

---- Result ---- 2
Module = Operations
Screen = AMISFU
Answer = The essential functions for a configuration tree are defined in the AFTREE screen, Function frame, Ess Qty. Field.
Question = How are the functions that are essential in a configuration tree defined?
----------------

---- Result ---- 3
Module = Operations
Screen = AMISFU
Answer = You will not be able to count on information to discriminate between an asset with full operational capacity from another that could be limited due to a lack of essential components.
Question = What if the essential functions are not defined in a configuration tree?
----------------

---- Result ---- 4
Module = Configuration
Screen = AFTREE
Answer = If the configuration tree is not defined, it will not be possible to instal

----

## **3  Define GenAI**

In [10]:
import oci 

compartment_id = "ocid1.compartment.oc1..aaaaaaaae3n6r6hrjipbap2hojicrsvkzatrtlwvsyrpyjd7wjnw4za3m75q"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)


def predict_genai(input_text):

    # Service endpoint
    endpoint = "https://generativeai.aiservice.us-chicago-1.oci.oraclecloud.com"

    generative_ai_client = oci.generative_ai.GenerativeAiClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

    prompts = [input_text]
    generate_text_detail = oci.generative_ai.models.GenerateTextDetails()
    generate_text_detail.prompts = prompts
    generate_text_detail.serving_mode = oci.generative_ai.models.OnDemandServingMode(model_id="cohere.command")
    # generate_text_detail.serving_mode = oci.generative_ai.models.DedicatedServingMode(endpoint_id="custom-model-endpoint") # for custom model from Dedicated AI Cluster
    generate_text_detail.compartment_id = compartment_id
    generate_text_detail.max_tokens = 400
    generate_text_detail.temperature = 0.75
    generate_text_detail.frequency_penalty = 1.0
    generate_text_detail.top_p = 0.7

    generate_text_response = generative_ai_client.generate_text(generate_text_detail)

    # Print result
    print("**************************Generate Texts Result**************************")
    #print(generate_text_response.data)
    print("Response from GenAI = " + generate_text_response.data.generated_texts[0][0].text)
    output_text = generate_text_response.data.generated_texts[0][0].text

    return output_text

## **4 Function to get Vector response and apply GenAI**

---

In [11]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client import AsyncQdrantClient, models

def function_2(question):
    
    qdrant = QdrantClient("207.211.188.211", port=6333) # 
    
    from sentence_transformers import SentenceTransformer
    encoder = SentenceTransformer('multi-qa-distilbert-cos-v1') # Model to create embeddings
    
    ## example Search on a sentence
    hits = qdrant.search(
        collection_name="collection_v3",
        query_vector=encoder.encode(question).tolist(),
        limit=10
    )
    
    #del list_of_answers
    list_of_answers = []

    x = 1
    for hit in hits:
        #print(hit.payload, "score:", hit.score)
        print("---- Result ---- " + str(x))
        output = hit.payload
        module = output['MODULE']
        screen = output['SCREEN']
        answer = output['ANSWER']
        questionx = output['QUESTION ']

        print("Module = " + module)
        print("Screen = " + screen)
        print("Answer = " + answer)
        print("Question = " + questionx)
        print("----------------")
        print()

        list_of_answers.append(answer)

        x += 1
    
    input_text = "Answer the following question: " + question + ". Base your answer on the following list of information " + str(list_of_answers)
    print(input_text)
    
    #apply Genai to list of answers + to answer the original question
    output_text = predict_genai(input_text)
        
    return list_of_answers, output_text


In [12]:
list_of_answers, output_text= function_2("QUSTIONOIOJO")

---- Result ---- 1
Module = Purchasing
Screen = SREQ
Answer = When a material request is defined as ‘type need - PRIO’, the status of the requisition can only be changed to status ‘A – APPROVED’  by someone at the ‘Prio Authorization level’ of that store, defined in STORED or by the Material Manager. 
Question = What if I use Type need ‘PRIO’ in SREQ?
----------------

---- Result ---- 2
Module = Purchasing
Screen = SREQ
Answer = When the external requisition is approved, the ‘Generate RFQ` button will be able, and 	you can push this button to create an automatic request for quotation that you can then 		manage on the PRFQ screen.
Question = How can I create a request for quotation from SREQ screen?
----------------

---- Result ---- 3
Module = Purchasing
Screen = PORD
Answer = In ILIAS application, you can define the responsibility of the importers in the arrangement of material shipments in a purchase order in the screen PORD, tab ‘General’, frame ‘Incoterm’.
Question = Where can I d

In [13]:
## question
# I want to change the value of the counter, please help me
#I want to view the ability status of all my assets i'm responsible for. how do I do that?

In [ ]:
desc = "Query Vector Database & Use GenAI"

with gr.Blocks() as demo: 

    #input 
    question = gr.Text(label="What are you looking for?")
   
    #output    
    list_of_answers = gr.Text(label="Most related Answers based on similar Question")
    output_text = gr.Text(label="Full answer")


    submit_btn = gr.Button("Search for your question")


gr.Interface(fn=function_2, inputs=question, outputs=[list_of_answers, output_text], title=desc).launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://b804f15d3174f1fc18.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


---- Result ---- 1
Module = Material Management
Screen = SMET
Answer = In the SMET screen, field Minimum and Maximum you can limit the value of a counter. As well as you can also limit the maximum deviation (variation) of a counter.
Question = How can I limit the value of a counter?
----------------

---- Result ---- 2
Module = Configuration
Screen = ASSETA
Answer = The counters of an Asset can be corrected on WREADD screen, filling the necessary 		information in the ‘Reading Characteristics’ and ‘List of counters for the Asset’ Frames. 
Question = Where can I correct the value of an Asset counter?
----------------

---- Result ---- 3
Module = Material Management
Screen = SMET
Answer = The counters are associated in the SMET screen through a common NSN for a group of assets that share said NSN.
Question = How can I associate a counter to an asset?
----------------

---- Result ---- 4
Module = Configuration
Screen = ASSETA
Answer = The counter new values of an Asset are recorded in WREA